In [1]:
%pylab inline

from PyTools import err, binner, nbins
from sklearn.cluster import DBSCAN
from scipy.optimize import leastsq
from string import join
from scipy.ndimage.filters import gaussian_filter
import pres_style, os

Populating the interactive namespace from numpy and matplotlib


In [2]:
def put_FITS_names(table):
    descr = table.dtype.descr
    for i in xrange(3): descr.pop(0)
    descr = [("input_file", "|S25")] + descr

    new_table = zeros(table.shape[0], dtype=descr)
    new_table["input_file"] = ["spSpec-{0:05d}-{1:04d}-{2:03d}.fit".format(r["mjd"], r["plate"], r["fiberID"]) for r in table]
    for f in new_table.dtype.names[1:]:
        new_table[f] = table[f]
    
    return new_table

par_lbs = [r"$M_\star$", r"$\left<\log{t_\star}\right>_M$", r"$\left<\log{t_\star}\right>_L$",
           r"$\left<\log{Z_\star/Z\odot}\right>_M$", r"$\left<\log{Z_\star/Z\odot}\right>_L$", r"$A_V$"]

if "paper_style" in dir():
    mnorm = 1e9
    od = os.path.expandvars("$HOME/Documents/Papers/photofit_near_paper/")
    par_lbs = [r"$\log{M_\star/M\odot}$", r"$\left<\log{t_\star}\right>_M$", r"$\left<\log{t_\star}\right>_L$",
               r"$\left<\log{Z_\star/Z\odot}\right>_M$", r"$\left<\log{Z_\star/Z\odot}\right>_L$", r"$A_V$",
               r"$A_V$", r"$z$"]
    res_lbs = [r"$\Delta\log{M_\star}$", r"$\Delta\left<\log{t_\star}\right>_M$",
               r"$\Delta\left<\log{t_\star}\right>_L$", r"$\Delta\left<\log{Z_\star}\right>_M$",
               r"$\Delta\left<\log{Z_\star}\right>_L$", r"$\Delta A_V$"]
    hatching = "/"*15
    gray = "0.3"
    ssag_cl = "0.2"
    sdss_cl = "#F6C141"
    sdss_sg = [sdss_cl, "#E8601C"]
    jpas_cl = "#90C987"
    jpas_sg = [jpas_cl, "#4EB265"]
    spec_cl = "#7BAFDE"
    spec_sg = [spec_cl, "#1965B0"]
    map_col = cm.viridis
else:
    mnorm = 1e8
    od = ""
    par_lbs = ["Mass (log/M$\odot$)", "Age (log/yr)", "Age (log/yr)", "Metallicity (log/Z$\odot$)",
               "Metallicity (log/Z$\odot$)", "Dust Extinction (mag)", "Redshift"]

In [ ]:
#READ TABLES

catalog = genfromtxt("catalog.txt", dtype=None, names=True)

g05 = genfromtxt("gal2005.txt", dtype=None, names=True)

match_cat_in_g05 = array([n in g05["input_file"] for n in catalog["input_file"]], dtype=bool)
cat_sort = argsort(catalog["input_file"][match_cat_in_g05])

cat = catalog[match_cat_in_g05]

In [ ]:
sed_dir = os.path.expandvars("$master/inputs/real_deal7/spec/")
flist = ["processed_SPEC_"+name.replace(".fit", ".txt") for name in catalog["input_file"]]
snr_wall, snr_CaHK, snr_CaII, snr_Zfea, npx_wall, npx_CaHK, npx_CaII, npx_Zfea = [], [], [], [], [], [], [], []
for sed_file in flist:
    wl, fl, sg = loadtxt(sed_dir+sed_file, usecols=(0,1,2), unpack=True)

    wall_mask = (fl>0.0)&(sg>0.0)
    CaHK_mask = wall_mask&(wl>=3920.0)&(wl<=3980.0)
    CaII_mask = wall_mask&(wl>=8400.0)&(wl<=8700.0)
    Zfea_mask = wall_mask&(wl>=6500.0)&(wl<=8000.0)

    npx_wall.append(wall_mask.sum())
    npx_CaHK.append(CaHK_mask.sum())
    npx_CaII.append(CaII_mask.sum())
    npx_Zfea.append(Zfea_mask.sum())

    snr_wall.append(median(fl[wall_mask]/sg[wall_mask]) if npx_wall[-1]!=0 else 0.0)
    snr_CaHK.append(median(fl[CaHK_mask]/sg[CaHK_mask]) if npx_CaHK[-1]!=0 else 0.0)
    snr_CaII.append(median(fl[CaII_mask]/sg[CaII_mask]) if npx_CaII[-1]!=0 else 0.0)
    snr_Zfea.append(median(fl[Zfea_mask]/sg[Zfea_mask]) if npx_Zfea[-1]!=0 else 0.0)

sig2noi = zeros(catalog.size, dtype=[("input_file", "|S25"), ("snr_CaHK", "<f8"), ("snr_CaII", "<f8"),
                                     ("snr_Zfea", "<f8"), ("snr_wall", "<f8"), ("npix_CaHK", "<i8"),
                                     ("npix_CaII", "<i8"), ("npix_Zfea", "<i8"), ("npix_wall", "<i8")])
fields = sig2noi.dtype.names
cols = [catalog["input_file"], snr_CaHK, snr_CaII, snr_Zfea, snr_wall, npx_CaHK, npx_CaII, npx_Zfea, npx_wall]
for field, column in zip(fields, cols): sig2noi[field] = column

In [22]:
savetxt("snr.txt", sig2noi, fmt="%25s"+"%10.2f"*4+"%10d"*4, header=("%23s"+"%10s"*8)%fields)

In [23]:
savetxt("gal2005.txt", gal2005[match_g05_in_cat][g05_sort], fmt="%25s"+"%12.4f"*3, header=("%23s"+"%12s"*3)%gal2005.dtype.names)

In [4]:
#CLEAN GALLAZZI TABLE

# remove null values
mask = where(gal2005["m_mode"] != -99.0)
gal2005 = gal2005[mask]

# convert log(Z) into log(Z/Zo)
gal2005["z_mode"] -= log10(0.02)

# homologate field names
gal2005 = put_FITS_names(gal2005)
gal2005.dtype.names = ("input_file", "M_mod", "log_t_L_mod", "log_Z_L_mod")

# match gal2005 & other tables & sort by input_file
match_g05_in_cat = array([n in catalog["input_file"] for n in gal2005["input_file"]], dtype=bool)


g05_sort = argsort(gal2005["input_file"][match_g05_in_cat])
